In [1]:
import pandas as pd
import numpy as np
import os
from pandasql import sqldf
from scipy import stats
import time
import networkx as nx
import simple_icd_10_cm as cm
import random

In [2]:
num_pats = 120000 # CHOOSE
dataset_loc = "..." # Insert path to preprocessed csv

# Additional preprocessing

bnf_df2 = pd.read_csv(dataset_loc)
bnf_list = bnf_df2["bnfchapter"].value_counts()[:50].index  # List of BNF chapters to use as nodes in network
pats = np.random.choice(bnf_df2['patid'].unique(), num_pats, replace=False) # Choose random patients to use
bnf_df = bnf_df2[bnf_df2['patid'].isin(pats)]
bnf_df['eventdate'] = pd.to_datetime(bnf_df['eventdate'])
bnf_df = bnf_df.reset_index(drop=True)

# Model

result = []
for bnf2 in bnf_list: # Cycle through bnf_list
    # Dataframe for all patients and include flag if they are prescribed with D2 within 5 years
    q = """
    SELECT *, event2_bnf IS NOT NULL FROM (
    SELECT a.*, b.bnfchapter AS event2_bnf, b.eventdate as event2_date
    FROM bnf_df AS a
    LEFT JOIN bnf_df AS b
    ON a.patid = b.patid
    AND b.eventdate > a.eventdate
    AND julianday(b.eventdate) - julianday(a.eventdate) < 1825
    AND b.bnfchapter = {}
    LIMIT 100000000) AS T
    LIMIT 100000000;
    """.format(bnf2)
    df = sqldf(q)
    
    # Convert eventdate column to datetime datatype
    df['eventdate'] = pd.to_datetime(df['eventdate'])
    
    for bnf1 in bnf_list:
        if bnf1 == bnf2:
            continue # Skip to next iteration if bnf1 == bnf2
        
        # Dataframe for all D1 patients, with those leading to D2 indicated
        D1_patients = df.loc[df['bnfchapter'] == bnf1]

        # Find earliest episode with D1 per patient
        D1_patients = D1_patients.loc[D1_patients.groupby('patid').eventdate.idxmin()]
        
        # Total D1 patients which lead to D2
        num_D1_to_D2 = D1_patients['event2_bnf IS NOT NULL'].sum()
        
        Total_D1_patients = len(D1)
        
        dx_pair_result = []
        dx_pair_result.append(bnf1)
        dx_pair_result.append(bnf2)
        dx_pair_result.append(Total_D1_patients)
        dx_pair_result.append(num_D1_to_D2)        
        result.append(dx_pair_result)
        
MainResult = pd.DataFrame(result)
print("Main model finished")

In [ ]:
# Create matched dataset
matched_rows = []
for row in bnf_df.itertuples(index=False, name='Pandas'):
    try: # Find rows similar to row in original dataframe subject to conditions
        matched_rows.append(bnf_df[
        (row[2] - pd.Timedelta(days = 30) < bnf_df['eventdate']) &
        (bnf_df['eventdate'] < row[2] + pd.Timedelta(days = 30)) &
        (bnf_df['gender'] == row[4]) &
        (row[5] - 5 < bnf_df['yob']) &
        (bnf_df['yob'] < row[5] + 5) &
        (bnf_df['gen_ethnicity_int'] == row[11]) &
        (bnf_df['bnfchapter'] != row[8]) &
        (bnf_df['patid'] != row[0])
        ].sample(1))
    except:
        # If no matches, add a blank row
        matched_rows.append(pd.DataFrame(np.array([-1,0,'2000-01-01',0,0,0,'NO MATCH',0,0,0,0,0],ndmin=2), columns = 
                                         ['patid','bnfcode', 'eventdate', 'num', 'gender', 'yob', 'gen_ethnicity', 
                                          'bnf', 'bnfchapter', 'bnfsection', 'bnfparagraph', 'gen_ethnicity_int']))

# Convert to dataframe
matched_df = pd.DataFrame(np.array(matched_rows).squeeze(), columns =
                                         ['patid','bnfcode', 'eventdate', 'num', 'gender', 'yob', 'gen_ethnicity', 
                                          'bnf', 'bnfchapter', 'bnfsection', 'bnfparagraph', 'gen_ethnicity_int'])
# Change column types
matched_df = matched_df.astype({'patid': 'int64', 'bnfcode': 'int64', 'num': 'int64', 'gender': 'int64',
             'yob': 'int64', 'bnfchapter': 'int64', 'bnfsection': 'int64',
             'bnfparagraph': 'int64', 'gen_ethnicity_int': 'int64'})
# Rename columns
matched_df = matched_df.rename(columns={"patid": "m_patid", "bnfcode": "m_bnfcode", "eventdate": "m_eventdate", "num": "m_num",
                                        "gender": "m_gender", "yob": "m_yob","gen_ethnicity": "m_gen_ethnicity", 
                                        "bnf": "m_bnf", "bnfchapter": "m_bnfchapter", "bnfsection": "m_bnfsection",
                          "bnfparagraph": "m_bnfparagraph", "gen_ethnicity_int": "m_gen_ethnicity_int"})
print("matched df constructed")

In [ ]:
# Count number of matched patients who are prescribed with D2 within 5 years

# Join original and matched dataframes
fulldf = pd.concat([bnf_df, matched_df], axis=1, join="inner")

# Convert date columns to datetime datatype
fulldf['eventdate'] = pd.to_datetime(fulldf['eventdate'])
fulldf['m_eventdate'] = pd.to_datetime(fulldf['m_eventdate'])

result = []
for bnf2 in bnf_list: # Cycle through bnf_list
    # Dataframe for all patients and include flag if they lead to D2
    q = """
    SELECT *, event2_bnf IS NOT NULL FROM (
    SELECT a.*, b.patid AS patid_dup, b.bnfchapter AS event2_bnf, b.eventdate as event2_date
    FROM fulldf AS a
    LEFT JOIN bnf_df AS b
    ON a.m_patid = b.patid
    AND b.eventdate > a.m_eventdate
    AND julianday(b.eventdate) - julianday(a.m_eventdate) < 1825
    AND b.bnfchapter = {}
    LIMIT 100000000) AS T
    LIMIT 100000000;
    """.format(bnf2)
    df = sqldf(q)
    
    # Convert epistart and end cols to datetime datatype
    df['eventdate'] = pd.to_datetime(df['eventdate'])
    
    for bnf1 in bnf_list:
        if bnf1 == bnf2:
            continue # Skip to next iteration if bnf1 == bnf2
        
        # Dataframe for all D1 patients with their matches and those leading to D2 indicated
        D1_patients = df.loc[df['bnfchapter'] == bnf1]

        # Find earliest episode with D1 per patient
        D1_patients = D1_patients.loc[D1_patients.groupby('patid').eventdate.idxmin()]
        
        # Filter out NO MATCH rows
        D1_patients = D1_patients[D1_patients["m_patid"]!=-1]

        # Total D1 patients which lead to D2
        num_D1_to_D2 = D1_patients['event2_bnf IS NOT NULL'].sum()
        
        dx_pair_result = []
        dx_pair_result.append(bnf1)
        dx_pair_result.append(bnf2)
        D1_V2 = D1[D1['m_bnfchapter']!=bnf2] # Filter out rows where matched BNF code == D2
        Total_D1_patients = len(D1_V2)
        dx_pair_result.append(Total_D1_patients)
        dx_pair_result.append(num_D1_to_D2)        
        result.append(dx_pair_result)
        
# Convert results to dataframe 
MatchResult = pd.DataFrame(result)   
MainResult = MainResult.rename(columns = {0:'D1', 1:'D2', 2:'Total_D1_patients', 3:'D1toD2'})
MatchResult = MatchResult.rename(columns = {0:'D1', 1:'D2', 2:'matchlen', 3:'matchtoD2'})

# Combine results into one df
MainResult = pd.concat([MainResult, MatchResult['matchlen']], axis=1)
MainResult = pd.concat([MainResult, MatchResult['matchtoD2']], axis=1)

In [ ]:
# Calculate relative risk as well as RR confidence intervals

def RelRisk(row):
    if row['matchtoD2'] > 0:
        return (row['D1toD2']/row['Total_D1_patients'])/(row['matchtoD2']/row['matchlen'])
    else:
        return 1
MainResult['RR'] = MainResult.apply(lambda row: RelRisk(row), axis=1)

def LogCI(row): # Calculate SE(log(RR))
    if row['matchtoD2'] == 0 or row['D1toD2'] == 0:
        return float("inf")
    else:
        return np.sqrt(1/row['D1toD2']-1/row['Total_D1_patients']+1/row['matchtoD2']-1/row['matchlen'])
MainResult['Log_CI'] = MainResult.apply(lambda row: LogCI(row), axis=1)

def CImin(row): # Lower bound for CI
    if row['RR']-4.26*row['Log_CI']>=0: # Change 4.26 to appropriate (Bonferroni-corrected) z-score
        return np.exp(np.log(row['RR']-4.26*row['Log_CI']))
    else:
        return 0
MainResult['CImin'] = MainResult.apply(lambda row: CImin(row), axis=1)

def CImax(row): # Upper bound for CI
    if row['RR']+4.26*row['Log_CI']>=0:
        return np.exp(np.log(row['RR']+4.26*row['Log_CI']))
    else:
        return 10
MainResult['CImax'] = MainResult.apply(lambda row: CImax(row), axis=1)

def CIover1(row): # Indicate if CI crosses over 1
    if row['CImin'] <= 1 and row['CImax'] <= 1:
        return 1
    elif row['CImin'] >= 1 and row['CImax'] >= 1:
        return 1
    else:
        return 0
MainResult['CIover1'] = MainResult.apply(lambda row: CIover1(row), axis=1)

def CIsize(row):
    if row['CImax'] == float("inf"):
        return 10
    else:
        return row['CImax'] - row['CImin']
MainResult['CIsize'] = MainResult.apply(lambda row: CIsize(row), axis=1)

# Save to csv
# MainResult.to_csv('FULL models/BNF/BNFSec{}Results.csv'.format(num_pats), index=False)

# Final result metrics to report
metrics = []
metrics.append(len(fulldf)) # Total dataset size
metrics.append(1-np.sum(matched_df['m_patid']==-1)/len(matched_df)) # Percentage of matched patients found
metrics.append(np.mean(MainResult['CIsize'])) # Av RR interval size
metrics.append(np.sum(MainResult['CIover1'])/len(MainResult)) # Percentage CIs that do not cross 1
# pd.DataFrame(metrics).to_csv('BNFSec{}Metrics.csv'.format(num_pats), index=False)

In [ ]:
# Compare results with another network

df1 = MainResult
df2 = pd.read_csv('...') # Load another network
# Compare adjacency matrices using various metrics
df1 = df1.rename(columns={'RR':"RR2"}) # Change column name so no clash
df2 = pd.concat([df1['RR2'], MainResult['RR']], axis=1, join="inner")
dists = []
dists.append(np.sqrt(np.sum(df2.apply(lambda row: (row['RR2']-row['RR'])**2 ,axis=1)))) # Euclidean
dists.append(np.sum(df2.apply(lambda row: abs(row['RR2']-row['RR']) ,axis=1))) # Manhattan
dists.append(1-np.sum(df2.apply(lambda row: min(row['RR2'],row['RR']),axis=1))/np.sum(df2.apply(lambda row: max(row['RR2'],row['RR']),axis=1))) # Weighted Jaccard
# pd.DataFrame(dists).to_csv('...'.format(num_pats), index=False) # Save distance metric results